This version is based on Timo's NLL_V3
changes:


*   truncnorm distribution
*   multivariant random walk



In [7]:
import scipy.stats
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import expon, truncexpon, truncnorm, nbinom, norm
import pandas as pd
import time
import torch
from torch import nn
from torch import distributions
from torch import rand
from torch import autograd
from torch import optim
import truncnormal

In [8]:
np.random.seed(seed=101)
torch.manual_seed(101)
torch.use_deterministic_algorithms(True)
dtype = torch.float64
device = torch.device("cpu")

In [9]:
data = pd.read_csv('covid19model.csv')
data

,date,hospit,serial_interval,delay_distr
0,2020-02-17,0,0.046535,1.300600e-02
1,2020-02-18,0,0.087065,3.004645e-02
2,2020-02-19,0,0.112061,4.467391e-02
3,2020-02-20,0,0.119346,5.547300e-02
4,2020-02-21,0,0.114540,6.242203e-02
...,...,...,...,...
402,2021-03-25,38,0.000000,2.817211e-32
403,2021-03-26,31,0.000000,2.349426e-32
404,2021-03-27,39,0.000000,1.959313e-32
405,2021-03-28,32,0.000000,1.633974e-32


In [10]:
def trunc_exponential(scale, upper):
    sample = torch.distributions.exponential.Exponential(1/scale).rsample()
    sample = sample/torch.tensor(1-torch.exp(-upper/scale))
    return sample
# torch.distributions.exponential.Exponential(1/scale).sample()/torch.tensor(1-torch.exp(-upper/scale))

def trunc_normal(mu, sigma, under, upper):
    distribution = torch.distributions.normal.Normal(loc=mu, scale=sigma, validate_args=None)
    normal_sample = distribution.rsample()
    cumulative = distribution.cdf(torch.tensor(upper)) - distribution.cdf(torch.tensor(under))
    return normal_sample/cumulative

# Initialization


In [11]:
cero = torch.tensor(0., requires_grad=False, device=device, dtype=dtype)
num_impute = 6
observed_daily_hospit = torch.tensor(data.hospit, requires_grad=False, device=device, dtype=dtype)
pi = torch.tensor(data.delay_distr, requires_grad=False, device=device, dtype=dtype)
serial_interval = torch.tensor(data.serial_interval, requires_grad=False, device=device, dtype=dtype)
population = torch.tensor(5793636, requires_grad=False, device=device, dtype=dtype)
num_observations = len(observed_daily_hospit)

## Initialize latent variables/parameters

In [18]:
torch.distributions.MultivariateNormal(torch.zeros(num_observations-1), sigma*torch.eye(num_observations-1)).rsample()

tensor([ 1.0733e+00, -3.1821e-01, -2.4436e-01, -6.8675e-01, -6.8496e-01,
         8.7369e-01, -3.3980e-02,  2.0174e-01, -1.4180e-01,  2.1514e-01,
        -5.2143e-01, -9.2417e-01,  1.2629e-01,  2.1507e-01, -1.7633e-02,
         1.5613e+00, -7.6018e-01,  2.8154e-01, -6.9298e-01,  1.2002e+00,
        -2.7572e-01, -1.4258e+00,  7.3912e-01,  1.9305e-01,  3.9020e-01,
        -5.9293e-02,  2.4745e-01, -1.1772e-01,  1.0063e+00, -2.0974e-01,
        -2.0275e-01, -9.9180e-02, -3.4494e-01, -1.5984e-01, -4.0807e-01,
         2.8705e-01,  2.1059e-01, -1.1559e+00, -4.1075e-01,  1.5147e-01,
         2.3461e-01, -1.1814e+00,  2.9631e-01,  9.5915e-01, -1.0209e+00,
         2.7373e-01,  2.6206e-01, -2.4661e-01,  1.7717e-01, -1.8821e-01,
        -1.2893e+00, -8.1639e-01,  1.2374e+00,  6.9343e-01,  6.5341e-01,
         5.0520e-02, -8.5176e-01, -1.7332e+00, -7.0140e-01,  2.1032e+00,
        -5.7433e-01,  7.8814e-01, -1.1000e+00,  1.0778e+00,  9.7963e-01,
         1.2395e+00, -9.9904e-01,  5.4068e-01, -5.4

In [20]:
tau= torch.tensor(np.random.exponential(1 / 0.03), requires_grad=True, device=device, dtype=dtype)
phi = torch.tensor(truncnorm.rvs((0 - 25) / 10, (np.inf - 25) / 10, loc=25, scale=10), requires_grad=True, device=device, dtype=dtype) # has to be positive, between 0-50 --> uniform # dispersion (shape) parameter for observations
R0 = torch.tensor(truncnorm.rvs((2 - 3.6) / 0.8, (5 - 3.6) / 0.8, loc=3.6, scale=0.8), requires_grad=True, device=device, dtype=dtype)  # probably gamma or inverse gamma distribution (compare to truncated normal) # initial reproduction number
alpha = torch.tensor(truncnorm.rvs((0 - 1/100) / 1/100, (5/100 - 1/100) / 1/100, loc=1/100, scale=1/100), requires_grad=True, device=device, dtype=dtype) # uniform distribution between (0-5%) # probability to get hospitalized
sigma = torch.tensor(truncnorm.rvs((0 - 0.1) / 0.3, (0.5 - 0.1) / 0.3, loc=0.1, scale=0.3), requires_grad=True, device=device, dtype=dtype)  # positive, tricky, gamma or inverse gamma, log normal  --> try something out, large sigma--> prone to overfitting # standart deviation of random walk step

epsilon_t = torch.zeros(num_observations, device=device)
diff_epsilon_t = torch.distributions.MultivariateNormal(torch.zeros(num_observations), sigma*torch.eye(num_observations)).rsample()
epsilon_t[0] = torch.distributions.Normal(cero, sigma.detach()).rsample()
for t in range(1, num_observations):
  epsilon_t[t] = epsilon_t[t - 1] + diff_epsilon_t[t]
#diff_epsilon_t.requires_grad_(True)
epsilon_t.requires_grad_(True)

tensor([ 0.0777,  0.2661,  0.2041,  0.0323,  0.1055,  0.4094,  0.3090,  0.6719,
         0.3415,  0.5796,  0.8001,  0.3693,  0.3362, -0.1237, -0.2820,  0.0731,
         0.7116,  0.2101,  0.1350,  0.1721,  0.3486,  0.8245,  0.8506,  1.1988,
         1.4345,  1.0071,  0.6316,  0.7394,  1.4771,  1.8781,  1.7981,  1.2003,
         0.8061,  0.5663,  0.1312,  0.6012,  0.9330,  1.0824,  0.4464,  0.3422,
        -0.0392,  0.1396,  0.2777,  0.4981,  0.4645,  0.2193,  0.5827,  0.1460,
        -0.3145, -0.7727, -0.5663, -1.1548, -1.0306, -1.0694, -1.3055, -1.3348,
        -1.4826, -1.5629, -1.5653, -1.8091, -1.4486, -1.7492, -1.7735, -1.5875,
        -1.6866, -1.9065, -1.8240, -2.1493, -2.5046, -2.2776, -2.8888, -2.2179,
        -2.2683, -2.5825, -2.5370, -2.3350, -2.3644, -2.4896, -2.0625, -1.8114,
        -1.1298, -0.9447, -1.3448, -1.0711, -0.9994, -0.4986, -0.9219, -1.1973,
        -1.0318, -0.5898, -0.4705, -0.3161, -0.4955, -0.3427, -0.4156, -0.5257,
        -0.4765, -1.1528, -1.5186, -1.90

This is a way to generate the initial params from pytorch distribution directly without truncation.
NOTE: Use either this code block below or above.

In [21]:
dist_tau_t = distributions.exponential.Exponential(torch.tensor([1/0.03], device=device))
#tau_t = dist_tau_t.sample()

dist_y = distributions.exponential.Exponential(tau)
#y = dist_y.sample()

dist_phi = truncnormal.TruncatedNormal(loc=torch.tensor([25], device=device), scale=torch.tensor([10], device=device), a= torch.tensor([0], device=device), b= torch.tensor([float('inf')], device=device))
#dist_phi = distributions.normal.Normal(loc=torch.tensor([25], device=device), scale=torch.tensor([10], device=device))
#phi = dist_phi.sample()

dist_R0 = truncnormal.TruncatedNormal(loc=torch.tensor([3.6], device=device), scale=torch.tensor([0.8], device=device), a= torch.tensor([2], device=device), b= torch.tensor([5], device=device))
#dist_R0 = distributions.normal.Normal(loc=torch.tensor([3.6], device=device), scale=torch.tensor([0.8], device=device))
#R0 = dist_R0.sample()

dist_alpha = truncnormal.TruncatedNormal(loc=torch.tensor([0.01], device=device), scale=torch.tensor([0.01], device=device), a= torch.tensor([0], device=device), b= torch.tensor([0.05], device=device))
#dist_alpha = distributions.normal.Normal(loc=torch.tensor([0.01], device=device), scale=torch.tensor([0.01], device=device))
#alpha = dist_alpha.sample()

dist_sigma = truncnormal.TruncatedNormal(loc=torch.tensor([0.1], device=device), scale=torch.tensor([0.3], device=device), a= torch.tensor([0], device=device), b= torch.tensor([0.5], device=device))
#dist_sigma = distributions.normal.Normal(loc=torch.tensor([0.1], device=device), scale=torch.tensor([0.3], device=device))
#sigma = dist_sigma.sample()

# Define Forward Pass

In [27]:
def calc_prior_loss(tau, phi, R0, alpha, sigma):
  # log likelihood wrt. our prior ("regularisation")
  # ll stands for log-likelihood
  ll = torch.tensor(0.0, device=device)

  #dist_tau_t = distributions.exponential.Exponential(torch.tensor([1/0.03]))
  #ll += dist_tau_t.log_prob(tau).item()

  #dist_y = distributions.exponential.Exponential(tau) #the parameter in the brasket should either be float or tensor, to avoid any inconvienience,
                                                      # I use everything as tensor. NOTE:tau_t is already a tensor.
  #ll += dist_y.log_prob(y).item()

  #dist_phi = distribution.normal.Normal(loc=torch.tensor([25]), scale=torch.tensor([10]))
  ll += dist_phi.log_prob(phi)

  #dist_R0 = distribution.normal.Normal(loc=torch.tensor([3.6]), scale=torch.tensor([0.8]))
  ll += dist_R0.log_prob(R0)

  #dist_alpha = distribution.normal.Normal(loc=torch.tensor([0.01]), scale=torch.tensor([0.01]))
  ll += dist_alpha.log_prob(alpha)

  #dist_sigma = distribution.normal.Normal(loc=torch.tensor([0.1]), scale=torch.tensor([0.3]))
  ll += dist_sigma.log_prob(sigma)

  return ll

In [23]:
def seed_init_infect(y):
  # Initialize newly_infected, cumulative_infected, St
  newly_infected = torch.zeros(num_observations, device=device, dtype=dtype)  # number of newly infected
  cumulative_infected = torch.zeros(num_observations, device=device)  # cumulative number of infected
  
  St = torch.zeros(num_observations, device=device)  # fraction of susceptible population
  # seed initial infection / impute first num_impute days
  newly_infected[0:num_impute] = y.clone()
  cumulative_infected[0] = 0.
  cumulative_infected[1:num_impute] = torch.cumsum(newly_infected[0:num_impute - 1].clone(), dim=0)
  St[0:num_impute] = torch.tensor([torch.maximum(population.clone() - x, torch.tensor(0)) / population for x in cumulative_infected[0:num_impute].clone()])
  return newly_infected, cumulative_infected, St

In [24]:
def calc_Rt(R0, epsilon_t, sigma, ll):
  # Initialize eta_t
  eta_t = torch.zeros(num_observations, device=device)  # transformed reproduction number

  # calculate Rt: the basic reproduction number
  # basic reproduction number as a latent random walk
  beta_0 = torch.log(R0)
  eta_t[0] = beta_0
  for t in range(1, num_observations):
      dist_epsilon_t = torch.distributions.Normal(epsilon_t[t - 1], sigma)
      dist_diff_epsilon_t =  torch.distributions.MultivariateNormal(torch.zeros(num_observations-1), sigma*torch.eye(num_observations-1))
      ll += dist_diff_epsilon_t.log_prob(diff_epsilon_t)
      #ll += dist_epsilon_t.log_prob(epsilon_t[t - 1])
  eta_t[1:num_observations] = beta_0 + epsilon_t[0:num_observations-1].clone()
  Rt = torch.exp(eta_t)
  return Rt, ll

In [25]:
def calc_infections(cumulative_infected, newly_infected, St, Rt):
  # Initialize effectively_infectious
  effectively_infectious = torch.zeros(num_observations, device=device)  # effective number of infectious individuals
  
  # calculate infections
  for t in range(num_impute, num_observations):
      # Update cumulative newly_infected
      cumulative_infected[t] = cumulative_infected[t - 1].clone() + newly_infected[t - 1].clone()
      # Adjusts for portion of pop that are susceptible
      St[t] = torch.maximum(population.clone() - cumulative_infected[t].clone(), cero) / population.clone()
      # effective number of infectous individuals
      ni_temp = newly_infected[:t].view(1, 1, -1).clone()
      si_temp = torch.flip(serial_interval, (0,))[-t:].view(1, 1, -1)
      effectively_infectious[t] = torch.nn.functional.conv1d(ni_temp, si_temp)
      
      newly_infected[t] = St[t].clone() * Rt[t].clone() * effectively_infectious[t].clone()
  return newly_infected

In [28]:
def calc_hospit(newly_infected, alpha):
  # Initialize expected_daily_hospit
  expected_daily_hospit = torch.zeros(num_observations, device=device)  # expected number of daily hospitalizations

  # calculate expected number of hospitalizations
  expected_daily_hospit[0] = (1e-15) * newly_infected[0].clone()
  for t in range(1, num_observations):
      ni_temp = newly_infected[:t].view(1, 1, -1)
      pi_temp = torch.flip(pi, (0,))[-t-1:-1].view(1, 1, -1)
      expected_daily_hospit[t] = torch.nn.functional.conv1d(ni_temp, pi_temp)
  expected_daily_hospit = alpha * expected_daily_hospit
  return expected_daily_hospit

In [29]:
def compare_results(expected_daily_hospit, phi, ll):
  # compare observed hospitalizations to model results
  # likelihood of the data wrt. to the model

  for i in range(0, num_observations):
      p = 1/(1+ expected_daily_hospit[i]/phi)
      dist = torch.distributions.negative_binomial.NegativeBinomial(phi, p-torch.tensor(2.225e-5))
      ll += dist.log_prob(observed_daily_hospit[i])
  return ll

In [30]:
def forward_pass():
  # Initialize y
  y = trunc_exponential(tau, 1000)

  # Calculate prior loss
  ll = calc_prior_loss(tau, phi, R0, alpha, sigma)
  
  # Seed initial infections
  newly_infected, cumulative_infected, St = seed_init_infect(y)
  
  # Calculate Rt & random walk loss 
  Rt, ll = calc_Rt(R0, epsilon_t, sigma, ll)
  
  # Calculate infections
  newly_infected = calc_infections(cumulative_infected, newly_infected, St, Rt)
  
  # Calculate expected hospitalizations
  expected_daily_hospit = calc_hospit(newly_infected, alpha)
  
  # Compare observed hospitalizations to model results
  ll = compare_results(expected_daily_hospit, phi, ll)
  return expected_daily_hospit, Rt, ll

# Optimization

In [31]:
learning_rate = 1e-12
epochs = 100
complete_time = time.time()

for k in range (epochs):
    start_time = time.time()
    decay = (1 - (k / (epochs*1e5))) ** 2
    learning_rate = learning_rate * decay

    # forward pass - calculate expected_daily_hospit
    expected_daily_hospit, Rt, ll = forward_pass()

    #backward pass
    loss = -ll
    loss.backward()

    if k%5 == 0:
      print(f'Time Step: {k}|| Loss: {loss},  R0:{R0}, grad: {R0.grad}, alpha: {alpha} grad: {alpha.grad}, phi: {phi} grad: {phi.grad}, sigma: {sigma} grad {sigma.grad}, epsilon_t.mean: {epsilon_t.mean()} grad.mean {epsilon_t.grad.mean()}')
      print("This Run:  %s seconds" % (time.time() - start_time))
    with torch.no_grad(): # this part is SGD. can also replace with loss.step
        tau -= learning_rate * tau.grad
        phi -= learning_rate * phi.grad 
        R0 -= learning_rate * R0.grad 
        alpha -= learning_rate * alpha.grad 
        sigma -= learning_rate * sigma.grad 
        epsilon_t -= learning_rate * epsilon_t.grad * 1e+8

        tau.grad = None
        phi.grad = None
        R0.grad = None
        alpha.grad = None
        sigma.grad = None
        epsilon_t.grad = None

    
    if k%100 == 0:    
      plt.plot(expected_daily_hospit.cpu().detach().numpy(), label='expected_daily_hospit')
      plt.plot(observed_daily_hospit.cpu().detach().numpy(), label='observed_daily_hospit')
      plt.legend()
      plt.show()


print("Complete Run:  %s seconds" % (time.time() - complete_time))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until


RuntimeError: ignored